<a href="https://colab.research.google.com/github/leo2991/leo2991.github.io/blob/main/%E3%80%8CWeb_Scraping_Guide_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

# Web scraping in Python

Scraping refers to extracting useful data from web pages which are written in a programming language called HTML. To scrap data from the HTML tree we first have to download the web page to our PC.

We will use the following packages to achieve the tasks in this lesson:
- [`requests`](http://docs.python-requests.org/en/master/)
- [`beautifoulsoup4`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#quick-start)

### Install the packages using pip

In [1]:
!pip install requests

In [2]:
!pip install beautifulsoup4

### Import the modules

In [3]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

## Basic concepts

### Fetch webpage contents using requests

To get everything about a webpage we use the `get` method from requests. There are many optional arguments it can take but the one main argument it takes is the url to the webpage you want retrieved.

In [4]:
URL = "https://github.com/requests/requests"

requests.get(URL)

<Response [200]>

The result of this method is a Response object. The number 200 is a [status code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). 200 is OK and it means no error.

In [5]:
requests.get(URL, {}).text

'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-iNnUjxnDZbx2GqneSSzGFrvmlaN/BjrGF5q/p8z9Fa0/c+MsD+geqNbi5m2pBphVcQ4hV3yIWFLvyreiyDwvkQ==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-color-modes-88d9d48f19c365bc761aa9de492cc616.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-kl1ZMaGw3nXKpJ7dv+lHQh/mCmIpQ1s7kPJMKZSZOcPvBy

To get the HTML as a string we use the `text` property of the Response object.

Before we go farther you should know that often you will get an error when accessing the webpage. There are many errors and even more causes for the error, but the most common cases are:
- You use a wrong URL.
- The website is down. To be sure this happens access it via browser.
- The website blocks bots and scraping agents. You can try to use browser looking UserAgent to fix this. If this happens investigate the `headers` parameter of the `get` method. It usually helps to use a plausible UserAgent but if it doesn't good luck trying to find a solution.

We can convert that text into either a BeautifoulSoup object.

#### Example 1

Create a beautifoul soup object.

In [6]:
web_page = bs4.BeautifulSoup(requests.get(URL, {}).text, "lxml")

In [7]:
web_page

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-color-modes-88d9d48f19c365bc761aa9de492cc616.css" integrity="sha512-iNnUjxnDZbx2GqneSSzGFrvmlaN/BjrGF5q/p8z9Fa0/c+MsD+geqNbi5m2pBphVcQ4hV3yIWFLvyreiyDwvkQ==" media="all" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-925d5931a1b0de75caa49eddbfe94742.css" integrity="sha512-kl1ZMaGw3nXKpJ7dv+

Web pages are trees of elements nested one inside the other.

For example:
- html
  - body
      - div
      - div
      - div
      
We say that body is a child of html and html is a parent of body, and that the 3 div are children of body. The 3 div are siblings. This terminology matters because the method names in bs4 follow it. 

Before you go scrapping open the website in Inspector View to see the nesting hierarchy of web page elements.

Generally all web pages have two main sections called `head` and `body`:
- `head` is where a lot of metadata lives
- `body` is what you seen on the screen and it contains all links, tables and images.

#### Example 2

Let's find the title of the web page we pulled using the `head` and `title` elements.

In [8]:
web_page.head.title

<title>GitHub - psf/requests: A simple, yet elegant HTTP library.</title>

We can navigate the tree by going element by element. You need to know the element names (html, head, div, span, p, a and so on) but don't worry if you don't. Look at the webpage in the inspector view in your browser and you can see the full path to the element of interest.

To get the text we need to use the `text` property of elements.

In [9]:
web_page.head.title.text

'GitHub - psf/requests: A simple, yet elegant HTTP library.'

#### Example 3

Let's go into the body of the github page we accessed.

In [10]:
web_page.body

<body class="logged-out env-production page-responsive">
<div class="position-relative js-header-wrapper ">
<a class="px-2 py-4 bg-blue text-white show-on-focus js-skip-to-content" href="#start-of-content">Skip to content</a>
<span class="progress-pjax-loader width-full js-pjax-loader-bar Progress position-fixed">
<span class="Progress-item progress-pjax-loader-bar " style="background-color: #79b8ff;width: 0%;"></span>
</span>
<header class="Header-old header-logged-out js-details-container Details position-relative f4 py-2" role="banner">
<div class="container-xl d-lg-flex flex-items-center p-responsive">
<div class="d-flex flex-justify-between flex-items-center">
<a aria-label="Homepage" class="mr-4" data-ga-click="(Logged out) Header, go to homepage, icon:logo-wordmark" href="https://github.com/">
<svg aria-hidden="true" class="octicon octicon-mark-github text-white" height="32" version="1.1" viewbox="0 0 16 16" width="32"><path d="M8 0C3.58 0 0 3.58 0 8c0 3.54 2.29 6.53 5.47 7.59.4

It is full of elements like `<a>` or `<ul>` or `<li>` or `<div>` or `<span>` etc.

The majority of that is noise to us because we want to find the numbers which describe this repository.

#### Example 4

Get the numbers we want.

When you open the inspector view you will see that 9 elements are in a parent-children relation before the element we care about which is `<ul class="numbers-summary">`.

We can use the approach we used above to reach it but there is a faster way using the `find_all` method:
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all

But before we jump into it you must learn about element attributes like `class="numbers-summary"`.

Element attributes are like a python dictionary and actually in bs4 they are a python dictionary. 

The attributes are standardized. There are many of them but the 3 most important ones for scrapping are:
- `id` which is a unique id of each element in the webpage
- `class` which is way to style multiple element in a same way
- `href` which is only valid for `<a>` elements and is the URL to which the element links to.

The result of find all is a list of web pages which are sub web pages of the original one and which satisfy some criteria. There are many such criteria and you can see them in the documentation but we use the `attrs` filter now. 

This filter in our example will give us back all web pages which start from an element named `<ul>` and which have an attribute called `class` and that attribute has the value `numbers-summary`. 

There is only a single such web page so we take the first element.

In [11]:
sub_web_page = web_page.find_all(name="ul", attrs={"class": "numbers-summary"})[0]

IndexError: ignored

To get the numbers we care about we need to filter our sub web page even more and again we will use `find_all` which is the work horse method you use 99% of the time.

In [12]:
sub_web_page.find_all("span")

NameError: ignored

We get a list of single element web pages which we need to transform to numbers.

We will use list comprehensions and do some string cleaning. 

Generally you will always need to do string cleaning when doing web scrapping so it's good to learn your regexes.

In [13]:
[int(wp.text.strip("\n ").replace(",", "")) 
    for wp in sub_web_page.find_all("span")]

NameError: ignored

#### Example 5

Get all the tags from the github page. The tags are `python`, `http`, `forhumans` etc.

In [14]:
URL = "https://github.com/requests/requests"

web_page_text = requests.get(URL).text

web_page = bs4.BeautifulSoup(web_page_text, "lxml")

web_page

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-color-modes-88d9d48f19c365bc761aa9de492cc616.css" integrity="sha512-iNnUjxnDZbx2GqneSSzGFrvmlaN/BjrGF5q/p8z9Fa0/c+MsD+geqNbi5m2pBphVcQ4hV3yIWFLvyreiyDwvkQ==" media="all" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-925d5931a1b0de75caa49eddbfe94742.css" integrity="sha512-kl1ZMaGw3nXKpJ7dv+

We are looking for the element `<div class="list-topics-container f6 mt-1">`.

Notice that class can have many entries, for example 3 as seen in this `div`.

When filtering by class we can use any single class, we don't have to list them all.

When we want to target a single element it is better to use `find` which has the same parameters as `find_all` but returns only a single sub web page.

Here we also use the `children` property for the first time.

In our list comprehension we filter by type and we remove all `NavigableString`s which are just string and not elements.

To understand the difference between a `NavigableString` and `Tag`(tag is synonimous for Element) look at this example.

```
    <div>
        I am some text and of type Navigable String.
        
        <a> I am a child I am of type Tag when I am an element but I am Navigable String when I am text </a>
   </div>
```

We keep only elements and access their text property and clean.

In [15]:
tags_elements = web_page.find(name="div", attrs={"class": "list-topics-container"})

tags_text = [elem.text.strip("\n ") 
             for elem in tags_elements.children 
             if type(elem) != bs4.NavigableString]

tags_text

AttributeError: ignored

#### Example 6

Achieve the same with less code.

We filter on `href` of all links in the webpage which satisfy our regular expression.

The `compile` method from `re` turns a regexp pattern into an object which can be used for matching.

If we passed only a string here it would look for exact matches to `href` not regexp matches.

The thing to learn here and from the whole lesson is that there is no rule on how to do web scrapping. Always use whatever works fastest and takes you least time to think of and type.

In [16]:
import re

[e.text.strip("\n ") for e in 
 web_page.find_all(name="a", 
                   attrs={"href": re.compile("/topics/.+")})]

['python',
 'http',
 'forhumans',
 'requests',
 'python-requests',
 'client',
 'humans',
 'cookies']

#### Example 7

We will now scrap a table element.

Tables in html are of form:
- table
    - tbody
        - tr (table row)
            - td (table column)
            - td
            - td
            
The element from which we start is:

`<table class="files js-navigation-container js-active-navigation-container" data-pjax="">`

As always use the inspector to see how the webpage is made and how we deconstruct it.

In [17]:
files_table = web_page.find(name="table", attrs = {"class": "files"}).tbody.children

files = []

for file_row in files_table:
    # ignore all navigable strings
    if type(file_row) == bs4.NavigableString:
        continue
        
    content = file_row.find(name="td", attrs={"class": "content"})\
                      .find(name="a")
        
    # If we didn't find the link ignore this element
    if content is None:
        continue
    
    # Get the href attribute
    href      = content.attrs["href"]    
    file_name = content.text.strip("\n ")
    
    files.append(("https://github.com" + href, file_name))
    
files

AttributeError: ignored

#### Example 8

Scrap multiple web pages.

We will find all files which are present in the `requests` github repository at root level and one level below it.

To do so we find all files and go into them following links.

In [18]:
def find_files(url):
    web_page = bs4.BeautifulSoup(requests.get(url).text, "lxml")
    
    files_table = web_page.find(name="table", attrs = {"class": "files"}).tbody.children

    files = []

    for file_row in files_table:
        # ignore all navigable strings
        if type(file_row) == bs4.NavigableString:
            continue

        content = file_row.find(name="td", attrs={"class": "content"})\
                          .find(name="a")

        # If we didn't find the link ignore this element
        if content is None:
            continue

        # Get the href attribute
        href      = content.attrs["href"]    
        file_name = content.text.strip("\n ")

        files.append(("https://github.com" + href, file_name))

    return files
    
for path, name in find_files(URL):
    print(name)
    
    # We wrap this in a try-catch block because we don't know what we might find
    # and we iteratively build our scrapper from error messages and the Inspector view
    # for example we do not check if we scrap a file or a directory and always assume directory
    # which crashes
    try:
        other_files = find_files(path)

        for path2, name2 in other_files:
            print(name2)
    except Exception as e:
        print("ERROR:" + str(e))
        print(path2)

AttributeError: ignored

## Full Example

This is an example scrapper for GDP from wikipedia per IMF.

The web page is located at:
- https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)

In the end we will have a pandas data frame of GDPs for each country in 2017.

In [19]:
WP_URL = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

# Fake the user agent so the web page thinks we access it as a regular human user
web_page = bs4.BeautifulSoup(requests.get(WP_URL, headers={
    "UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"
}).text, "lxml")

imf_table = web_page.find_all(name="table", attrs={"class": "wikitable"})[0]

# Get the column names of our dataframe. 
# `children` is an iterator and to index it we must first convert it to a list.
columns = list(imf_table.tbody.children)[0]
columns = [elem.text.strip("\n ") 
           for elem in columns 
           if type(elem) != bs4.NavigableString]

rows = []

for i, row in enumerate(imf_table.tbody.find_all("tr")):
    # Skip the header
    if i <= 1 or type(row) == bs4.NavigableString:
        continue

    tds = row.find_all("td")
    
    rank         = tds[0].text
    country_name = tds[1].text
    gdp          = tds[2].text
    
    rows.append((rank, country_name, gdp))
    
data_frame = pd.DataFrame(rows, columns=columns)

data_frame.head()

IndexError: ignored

Our data frame is full of unclean data and all are of type object.

Our next step is to clean our data.

In [20]:
import re

def clean_country(c):
    try:
        return re.findall(pattern="(.+)\[.+", string=c)[0].strip("\xa0")
    except:
        return c
    
# One call removes one set of brackets [] so two calls to fix EU if EU is fixable...
data_frame.Country = data_frame.Country.apply(clean_country)
data_frame.Country = data_frame.Country.apply(clean_country)
                    
data_frame.rename({"GDP(US$MM)": "GDP"}, inplace=True, axis=1)
data_frame.GDP = data_frame.GDP.apply(lambda gdp: gdp.replace("\n", "").replace(",", ""))

# Fix 77,460/Na for Syria
data_frame.GDP.replace("77460/Na", "77460", inplace=True)

data_frame.GDP = data_frame.GDP.astype(np.float64)

NameError: ignored

In [21]:
data_frame.info()

NameError: ignored

In [22]:
data_frame.head()

NameError: ignored

## Your very own scrapper

Pick a website of your choice and scrap the data into a pandas dataframe and clean it afterwards.